In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 32.7 MB/s 
     |████████████████████████████████| 212 kB 59.7 MB/s 


In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 575 kB 14.3 MB/s 


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 29.3 MB/s 
     |████████████████████████████████| 182 kB 74.1 MB/s 
     |████████████████████████████████| 7.6 MB 60.4 MB/s 


In [ ]:
import torch
import pandas as pd
import numpy as np
import shap
from keras_preprocessing.sequence import pad_sequences
from transformers import BertTokenizer
import scipy as sp
import transformers
import torch.nn as nn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!nvidia-smi

Fri Dec  2 23:01:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    27W /  70W |      3MiB / 15109MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model=torch.load(r'/content/drive/MyDrive/Model1.pt',map_location=torch.device('cuda'))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
isinstance(model, nn.DataParallel)

True

In [ ]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  return class_name

In [ ]:
def predictor(x):
    probas = model(**tokenizer(x, return_tensors="pt", padding=True))[0].softmax(1).detach().cpu().numpy()
    # logits = model(**tokenizer(x, return_tensors="pt", padding=True)).logits
    # predicted_class_id = logits.argmax().item()
    # classifications = ['Introduction','Depression','Grey Area']
    # classification = logits.argmax().item()
    # class_name = classifications[classification]
    # print(class_name)
    val = sp.special.logit(probas[:,1])
    return val

def f_batch(x):
    val = np.array([])
    for i in x:
      val = np.append(val, predictor(i))
    return val



In [ ]:
explainer_bert = shap.Explainer(f_batch, tokenizer)

In [ ]:
def explain(data):
  test = { 'text': [data]}
  shap_values = explainer_bert(test)
  print(adapter(data))
  shap.plots.text(shap_values[0])


In [ ]:
explain('I was born and grew up in NewYork with my grandparents')

Introduction


In [ ]:
explain('I am having episodes of severe depression since I was a teenager')

Depression


In [ ]:
explain('I like cooking as a hobby')

Introduction


In [ ]:
explain('I feel pretty low these days, it feels like my life is a black-hole')

Depression


In [ ]:
explain('Queen Elizabeth of britain passed away recently')

Grey Area


In [ ]:
explain('I feel really blue sometimes but that is okay')

Grey Area


In [ ]:
explain('I feel tired all the time even if I am getting enough sleep')

Depression


In [ ]:
explain('Sprinng is such a happy weather, everything is just perfect')

  0%|          | 0/210 [00:00<?, ?it/s]

Grey Area


In [ ]:
explain('Poetry and art is the language you speak')

Introduction


In [ ]:
explain('Beauty and the Beast movie was made by Disney')

Introduction


In [ ]:
explain('Sometimes I have lot of work to do which makes me feel burn out,I wish I had chosen different major')

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 2it [00:12, 12.01s/it]               

Depression


In [ ]:
explain("sometimes I don't want to do anthing, I just sit in a corner and cry")

  0%|          | 0/420 [00:00<?, ?it/s]

Depression


In [ ]:
explain("sometimes all you need is a little hope to go through your day")

Grey Area


In [ ]:
explain("I was diagnosed with tumor when I was 16.")

Introduction


In [ ]:
explain("Sunsets on beaches are love!!")

Introduction


In [ ]:
explain("Are you a dog person or a cat person?")

  0%|          | 0/132 [00:00<?, ?it/s]

Introduction


In [ ]:
explain("There is a really cute cat in my hostel")

Introduction


In [ ]:
explain("Roses are red,violets are blue")

Grey Area


In [ ]:
explain("I get anxious around people sometimes")

Depression


In [ ]:
explain("My last psychiatrist was a really compassionate doctor who actually listened to me")

Grey Area


In [ ]:
explain("I am going as a fashion designer for cultural meet fashion show")

Introduction


In [ ]:
explain("I would rather be sad than feel nothing at all")

Depression


In [ ]:
explain("Milk and Sugar is needed to make cake")

Grey Area


In [ ]:
explain("There was a point when I was really suicidal")

Depression


In [ ]:
explain("I don't know what to do, I just want to die and end everything")

  0%|          | 0/342 [00:00<?, ?it/s]

Depression


In [ ]:
explain("I called suicide prevention helpline once beacuse I thought I will kill myself")

  0%|          | 0/306 [00:00<?, ?it/s]

Partition explainer: 2it [00:10, 10.30s/it]               

Depression


In [ ]:
explain("I have happy emotions all the time")

Depression
